In [26]:
import pandas as pd
import numpy as np
import plotly.express as px

from Project.Database import Db

In [7]:
year1, meta = Db.load_data(meta=True, hourly=True, year= 1)
meta

,Subsystem,Measurement_Location,Parameter,Description,Units,Aggregation_Method,min_value,max_value,Measurement_Floor,Load_Match,Status_Match,Elec_Match,Consumer_Match
Unnamed: 0,,,,,,,,,,,,,
DHW_ClothesWasherColdFlow,DHW,Utility,Flow_Water,The cumulative volume of cold water flowing in...,Gallons,Hour value,0.0,33.0646,1stFloor,NaN,NaN,NaN,<NA>
DHW_ClothesWasherHotFlow,DHW,Utility,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0,201.6969,1stFloor,NaN,NaN,NaN,<NA>
DHW_DishwasherHotFlow,DHW,Kitchen,Flow_Water,The cumulative volume of hot water flowing int...,Gallons,Hour value,0.0,4.6367,1stFloor,NaN,NaN,NaN,<NA>
DHW_HeatPumpWaterHeaterEnergyTotal,DHW,Basement,Energy_Electrical,Cumulative energy consumption by heat pump wat...,Wh,Hour value,0.0,188169.7774,Basement,NaN,NaN,NaN,<NA>
DHW_HeatPumpWaterHeaterPowerTotal,DHW,Basement,Power_Electrical,Instantaneous power consumption by heat pump w...,W,Average,0.0,3556.3887,Basement,NaN,Load_StatusHeatPumpWaterHeater,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Load_StatusRefrigerator,Loads,Kitchen,Status_OnOff,Number to indicate whether the refrigerator is...,BinaryStatus,Average,0.0,1.0,1stFloor,Load_RefrigeratorPowerWithStandby,NaN,Elec_PowerRefrigerator,Load_RefrigeratorPowerWithStandby
Load_StatusMicrowave,Loads,Kitchen,Status_OnOff,Number to indicate whether the microwave oven ...,BinaryStatus,Average,0.0,1.0,1stFloor,Load_MicrowavePowerWithStandby,NaN,Elec_PowerMicrowave,Load_MicrowavePowerWithStandby
Load_StatusClothesWasher,Loads,Utility,Status_OnOff,Number to indicate whether clothes washer is a...,BinaryStatus,Average,0.0,1.0,1stFloor,Load_ClothesWasherPowerWithStandby,NaN,Elec_PowerClothesWasher,Load_ClothesWasherPowerWithStandby


In [8]:
year2, meta = Db.load_data(meta=True, hourly=True, year= 2)

In [14]:
consumption_condition = (lambda self: (
        (self["Units"] == "W") &
        self["Description"].str.contains("power consumption" or "used") &
        ~(self.index == "Elec_PowerMicrowave") &
        ~(self.index == "Elec_PowerRefrigerator") &
        ~(self.index == "Elec_PowerClothesWasher") &
        ~self.index.str.contains("Elec_[\w]+HVAC") &
        (~self.index.str.contains("DHW_") |
         self.index.str.contains("DHW_[\w]+Total")) &
        (~self.index.str.contains("Load_") |
         ~self.index.str.contains("Load_Microwave") &
         self.index.str.contains("Load_[\w]+Standby"))))

consumption = meta.loc[consumption_condition].index.tolist()

production = meta.loc[(meta['Parameter'] == "Power_Electrical") & (
    meta["Description"].str.contains("produced"))].index.tolist()
production

['PV_Watts3PhTotalW3PhT1', 'PV_Watts3PhTotalW3PhT2']

In [32]:
year1_consumption_production = year2[["Timestamp"] + consumption + production].copy()
year1_consumption_production

,Timestamp,Load_ClothesWasherPowerWithStandby,Load_RefrigeratorPowerWithStandby,Elec_PowerDryer1of2,Elec_PowerDryer2of2,Elec_PowerHRV,Elec_PowerHeatPumpWH1of2,Elec_PowerHeatPumpWH2of2,Elec_PowerMakeUpAirDamper,Elec_PowerLights2ndFloor,...,Elec_PowerParentBDownstairs,Elec_PowerPlugsInstKitD,Elec_PowerPlugsDRB,Elec_PowerPlugsAttic,DHW_HeatPumpWaterHeaterPowerTotal,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierPower,PV_Watts3PhTotalW3PhT1,PV_Watts3PhTotalW3PhT2
0,2015-02-01 00:00:00,2.9468,56.313900,0.0,0.0,59.534457,0.0,0.000000,0.0,0.000000,...,0.000000,43.048599,0.0,5.690366,6.920000,175.449169,1045.817746,4.498593,9.543753,8.952575
1,2015-02-01 01:00:00,2.9468,32.780866,0.0,0.0,64.442596,0.0,138.125886,0.0,0.000000,...,0.000000,46.010456,0.0,5.761672,280.403772,391.606267,1042.026917,4.442783,9.469817,8.861749
2,2015-02-01 02:00:00,2.9468,53.040107,0.0,0.0,64.487108,0.0,0.000000,0.0,0.000000,...,0.000000,45.938416,0.0,5.762010,6.920000,388.484683,1023.145217,4.444083,9.497751,8.886896
3,2015-02-01 03:00:00,2.9468,31.281387,0.0,0.0,64.560409,0.0,0.000000,0.0,0.000000,...,0.000000,45.672843,0.0,5.775810,6.920000,175.792833,1066.177350,4.467983,9.552120,8.949422
4,2015-02-01 04:00:00,2.9468,49.633387,0.0,0.0,64.338450,0.0,0.000000,0.0,0.000000,...,0.000000,45.845817,0.0,5.779499,6.920000,371.120983,1081.112783,4.466550,9.558916,8.965035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2016-01-31 19:00:00,2.9468,47.464094,0.0,0.0,45.825132,0.0,172.752848,0.0,70.120384,...,73.717998,47.922261,0.0,5.765019,349.196362,11.229733,22.489150,4.243500,9.158374,8.551447
8756,2016-01-31 20:00:00,2.9468,40.535826,0.0,0.0,46.016167,0.0,0.000000,0.0,56.003880,...,73.984977,47.925715,0.0,5.778190,6.920000,11.366567,22.483300,4.275667,9.281892,8.658388
8757,2016-01-31 21:00:00,2.9468,46.112473,0.0,0.0,45.728363,0.0,157.876316,0.0,44.115827,...,74.316919,48.009035,0.0,5.813705,319.558632,11.448617,22.509617,4.297500,9.382447,8.766212
8758,2016-01-31 22:00:00,2.9468,39.756467,0.0,0.0,46.169482,0.0,23.510936,0.0,96.432486,...,0.000000,48.076756,0.0,5.804656,53.365206,11.535333,22.481850,4.327800,9.413440,8.795712


In [33]:
year1_consumption_production['Summed_consumption'] = year1_consumption_production[consumption].sum(1)
year1_consumption_production['Summed_production'] = year1_consumption_production[production].sum(1)

In [34]:
year1_consumption_production['Energy_balance'] = year1_consumption_production['Summed_production'] - year1_consumption_production['Summed_consumption']
year1_consumption_production.loc[:, ['Timestamp', 'Energy_balance']]

,Timestamp,Energy_balance
0,2015-02-01 00:00:00,-1978.212098
1,2015-02-01 01:00:00,-2608.431001
2,2015-02-01 02:00:00,-2195.482013
3,2015-02-01 03:00:00,-2005.284492
4,2015-02-01 04:00:00,-2234.131743
...,...,...
8755,2016-01-31 19:00:00,-1358.479068
8756,2016-01-31 20:00:00,-810.352392
8757,2016-01-31 21:00:00,-1637.757682
8758,2016-01-31 22:00:00,-897.870625


In [35]:
fig = px.line(year1_consumption_production, x="Timestamp", y="Energy_balance")
fig.show()